Notas:
- Debería ordenarlo según: "2200, O'Higgins, Autonomous City of Buenos Aires"

In [1]:
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopy.geocoders import Nominatim
import re
import requests
import json
import time
from config.config_jupyter import set_wd
set_wd()

c:\Users\marti\OneDrive\Documentos\Trabajos Indec\Scrap alquileres caba


In [2]:
df = pd.read_parquet("output/zonaprop_vivienda.pkt")

In [3]:
df['Address']

0                           O'higgins al 2200
1       Tte. Gral. Juan Domingo Perón al 4300
2                           AV. SANTA FE 2100
3                        Av. Congreso al 1500
4                 Av Chenaut al 1700 Piso 10º
                        ...                  
9390                              French 3000
9391                   Teodoro Garcia al 2800
9392                             Peña 2000 2°
9393                             Avellaneda 0
9394                            Esmeralda 629
Name: Address, Length: 9395, dtype: object

In [10]:
def rearrange_address(address):
    # Use regular expression to find the number
    match = re.search(r'\d+', address)
    
    if match:
        number = match.group()  # Extract the number
        text = address.replace(number, '').strip()  # Remove the number from the address and strip extra spaces
        return f"{number}, {text}, Autonomous City of Buenos Aires"
    else:
        return None  # If no number is found, return None or handle accordingly
    
df["Address"] = (df['Address'].str.lower()
 .str.replace('al ', '')
 .str.split('piso ').str[0]
 .apply(lambda x: x.rsplit('°', 1)[0] if '°' in x else x)
 .apply(lambda x: rearrange_address(x))
 .str.split('entre ').str[0]
 .str.split('- ').str[0]
)

In [5]:
def get_coordinates(address):
    # Replace spaces with + for URL encoding
    address = address.replace(' ', '+')
    
    # Construct the API endpoint URL
    url = f"https://nominatim.openstreetmap.org/search?q={address}&format=jsonv2"
    
    # Send a GET request to the API
    response = requests.get(url)
    
    # Parse the JSON response
    data = json.loads(response.text)
    
    # Get the first result
    first_result = data[0] if data else None
    
    # Extract the latitude and longitude values
    lat_long = [first_result['lat'], first_result['lon']] if first_result else None
    
    return lat_long


In [12]:
get_coordinates(df["Address"][2])

['-34.595686', '-58.3972726']

In [13]:
# Más o menos tarda entre 3 y 4 días en hacerlo
buenos_aires_map = folium.Map(location=[-34.6037, -58.3816], zoom_start=13)

mc = MarkerCluster()
# for idx, row in daytime_robberies.iterrows():
#     if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
# mc.add_child(Marker(get_coordinates(df["Address"][2])))
for address in df["Address"]:
    time.sleep(1)
    try:
        mc.add_child(Marker(get_coordinates(address)))
    except Exception:
        pass
buenos_aires_map.add_child(mc)
# Display the map
buenos_aires_map